In [56]:
val movieDf = spark.read.format("csv")
                .option("header", true)
                .option("inferSchema", true)
                .option("delimitter", ",")
                .load("hdfs://localhost:9000/ml-latest-small/movies.csv")
movieDf.printSchema()
movieDf.show(2)


root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
|      2|  Jumanji (1995)|Adventure|Childre...|
+-------+----------------+--------------------+
only showing top 2 rows



movieDf: org.apache.spark.sql.DataFrame = [movieId: int, title: string ... 1 more field]


In [57]:
// create schema

import org.apache.spark.sql.types.{StringType, StructType, DoubleType, IntegerType, LongType, StructField}
val RatingSchema = StructType(
                                List(
                                    StructField("userId", IntegerType, true),
                                    StructField("movieId", IntegerType, true),
                                    StructField("rating", DoubleType, true),
                                    StructField("timestamp", LongType, true)
                                    )
                            )
                                   

import org.apache.spark.sql.types.{StringType, StructType, DoubleType, IntegerType, LongType, StructField}
RatingSchema: org.apache.spark.sql.types.StructType = StructType(StructField(userId,IntegerType,true), StructField(movieId,IntegerType,true), StructField(rating,DoubleType,true), StructField(timestamp,LongType,true))


In [58]:
val ratingDf = spark.read.format("csv")
                .option("header", true)
                .option("delimitter", ",")
                .schema(RatingSchema)
                .load("hdfs://localhost:9000/ml-latest-small/ratings.csv")
ratingDf.printSchema()
ratingDf.show(2)


root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
+------+-------+------+---------+
only showing top 2 rows



ratingDf: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 2 more fields]


In [59]:
import org.apache.spark.sql.functions.{col}

import org.apache.spark.sql.functions.col


In [60]:
val df = ratingDf.filter(ratingDf("rating") > 1.0)
df.count()

df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: int, movieId: int ... 2 more fields]
res31: Long = 96655


In [61]:
val df = ratingDf.filter($"rating" > 1.0)
df.count()

df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: int, movieId: int ... 2 more fields]
res32: Long = 96655


In [62]:
val df = ratingDf.filter('rating > 1.0)
df.count()

df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: int, movieId: int ... 2 more fields]
res33: Long = 96655


In [64]:
import org.apache.spark.sql.functions.{count,avg}

import org.apache.spark.sql.functions.{count, avg}


In [65]:
val popularDf = ratingDf
                        .groupBy($"movieId")
                        .agg(count($"userId").alias("total_ratings"))

popularDf: org.apache.spark.sql.DataFrame = [movieId: int, total_ratings: bigint]


In [67]:
popularDf.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- total_ratings: long (nullable = false)



In [68]:
popularDf.show(2)

+-------+-------------+
|movieId|total_ratings|
+-------+-------------+
|   1580|          165|
|   2366|           25|
+-------+-------------+
only showing top 2 rows



In [72]:
movieDf.createOrReplaceTempView("movies")

In [73]:
val df = spark.sql("SELECT * FROM movies")
df.printSchema()
df.show(2)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
|      2|  Jumanji (1995)|Adventure|Childre...|
+-------+----------------+--------------------+
only showing top 2 rows



df: org.apache.spark.sql.DataFrame = [movieId: int, title: string ... 1 more field]


In [74]:
df.explain()

== Physical Plan ==
FileScan csv [movieId#381,title#382,genres#383] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://localhost:9000/ml-latest-small/movies.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<movieId:int,title:string,genres:string>




In [75]:
ratingDf.createOrReplaceTempView("ratings")

In [76]:
val df = spark.sql("SELECT * FROM ratings")
df.printSchema()
df.show(2)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
+------+-------+------+---------+
only showing top 2 rows



df: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 2 more fields]


In [12]:
df.explain()

== Physical Plan ==
FileScan csv [userId#57,movieId#58,rating#59,timestamp#60L] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://localhost:9000/ml-latest-small/ratings.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<userId:int,movieId:int,rating:double,timestamp:bigint>




In [13]:
val df2 = spark.sql("SELECT movieId, upper(title), genres from movies LIMIT 5")

df2: org.apache.spark.sql.DataFrame = [movieId: int, upper(title): string ... 1 more field]


In [14]:
df2.show()

+-------+--------------------+--------------------+
|movieId|        upper(title)|              genres|
+-------+--------------------+--------------------+
|      1|    TOY STORY (1995)|Adventure|Animati...|
|      2|      JUMANJI (1995)|Adventure|Childre...|
|      3|GRUMPIER OLD MEN ...|      Comedy|Romance|
|      4|WAITING TO EXHALE...|Comedy|Drama|Romance|
|      5|FATHER OF THE BRI...|              Comedy|
+-------+--------------------+--------------------+



In [18]:
val df3 = spark.sql(" SELECT DISTINCT(rating) FROM ratings ORDER BY rating")

df3: org.apache.spark.sql.DataFrame = [rating: double]


In [19]:
df3.show()

+------+
|rating|
+------+
|   0.5|
|   1.0|
|   1.5|
|   2.0|
|   2.5|
|   3.0|
|   3.5|
|   4.0|
|   4.5|
|   5.0|
+------+



In [23]:
val df4 = spark.sql(" SELECT DISTINCT(rating) FROM ratings ORDER BY rating DESC")

df4: org.apache.spark.sql.DataFrame = [rating: double]


In [24]:
df4.show()

+------+
|rating|
+------+
|   5.0|
|   4.5|
|   4.0|
|   3.5|
|   3.0|
|   2.5|
|   2.0|
|   1.5|
|   1.0|
|   0.5|
+------+



In [25]:
val df5 = spark.sql("SELECT rating from ratings WHERE rating >= 4 and rating <=5")

df5: org.apache.spark.sql.DataFrame = [rating: double]


In [26]:
df5.show()

+------+
|rating|
+------+
|   4.0|
|   4.0|
|   4.0|
|   5.0|
|   5.0|
|   5.0|
|   4.0|
|   5.0|
|   5.0|
|   5.0|
|   5.0|
|   5.0|
|   4.0|
|   5.0|
|   5.0|
|   4.0|
|   4.0|
|   5.0|
|   4.0|
|   4.0|
+------+
only showing top 20 rows



In [48]:
val mostPopularMoviesDf= spark.sql("""SELECT movieId, COUNT(userId) as total_ratings, AVG(rating) as avg_ratings 
                    FROM ratings  
                    WHERE rating > 1.0
                    GROUP BY movieId 
                    HAVING avg_ratings >= 3.5 and total_ratings >= 100
                    ORDER BY avg_ratings DESC
                    """)

mostPopularMoviesDf: org.apache.spark.sql.DataFrame = [movieId: int, total_ratings: bigint ... 1 more field]


In [50]:
mostPopularMoviesDf.show()

+-------+-------------+------------------+
|movieId|total_ratings|       avg_ratings|
+-------+-------------+------------------+
|    318|          315| 4.450793650793651|
|    527|          213| 4.335680751173709|
|   2959|          215| 4.325581395348837|
|   1221|          127| 4.311023622047244|
|   2571|          269| 4.308550185873606|
|    858|          191| 4.306282722513089|
|    296|          299| 4.285953177257525|
|  48516|          106| 4.283018867924528|
|     50|          202|  4.26980198019802|
|    260|          249| 4.259036144578313|
|   1198|          197| 4.258883248730965|
|   1213|          126|              4.25|
|   4973|          118| 4.241525423728813|
|  58559|          149| 4.238255033557047|
|   1196|          210| 4.233333333333333|
|   1197|          142| 4.232394366197183|
|   1193|          132|4.2272727272727275|
|   1089|          130| 4.226923076923077|
|   1208|          107| 4.219626168224299|
|   2329|          129| 4.217054263565892|
+-------+--

In [77]:
mostPopularMoviesDf.createOrReplaceTempView("most_popular_movies")

In [80]:
val df = spark.sql("""
                    SELECT most_popular_movies.movieId, title, avg_ratings, total_ratings 
                    FROM most_popular_movies 
                    INNER JOIN movies on movies.movieId = most_popular_movies.movieId
                    ORDER BY total_ratings DESC
                    """)
df.show(5)

+-------+--------------------+-----------------+-------------+
|movieId|               title|      avg_ratings|total_ratings|
+-------+--------------------+-----------------+-------------+
|    356| Forrest Gump (1994)|4.185015290519877|          327|
|    318|Shawshank Redempt...|4.450793650793651|          315|
|    296| Pulp Fiction (1994)|4.285953177257525|          299|
|    593|Silence of the La...|4.212727272727273|          275|
|   2571|  Matrix, The (1999)|4.308550185873606|          269|
+-------+--------------------+-----------------+-------------+
only showing top 5 rows



df: org.apache.spark.sql.DataFrame = [movieId: int, title: string ... 2 more fields]


In [83]:
val df = spark.sql("""
                    SELECT mpm.movieId, title, avg_ratings, total_ratings 
                    FROM most_popular_movies mpm
                    INNER JOIN movies m on m.movieId = mpm.movieId
                    ORDER BY total_ratings DESC
                    """)
df.show(5)

+-------+--------------------+-----------------+-------------+
|movieId|               title|      avg_ratings|total_ratings|
+-------+--------------------+-----------------+-------------+
|    356| Forrest Gump (1994)|4.185015290519877|          327|
|    318|Shawshank Redempt...|4.450793650793651|          315|
|    296| Pulp Fiction (1994)|4.285953177257525|          299|
|    593|Silence of the La...|4.212727272727273|          275|
|   2571|  Matrix, The (1999)|4.308550185873606|          269|
+-------+--------------------+-----------------+-------------+
only showing top 5 rows



df: org.apache.spark.sql.DataFrame = [movieId: int, title: string ... 2 more fields]


In [88]:
df.rdd.getNumPartitions

res55: Int = 77


In [89]:
df.cache()

res56: df.type = [movieId: int, title: string ... 2 more fields]


In [90]:
mostPopularMoviesDf.cache()

res57: mostPopularMoviesDf.type = [movieId: int, total_ratings: bigint ... 1 more field]


In [92]:
mostPopularMoviesDf.rdd.getNumPartitions

res59: Int = 113


In [93]:
val rdd6 = mostPopularMoviesDf.coalesce(1)

rdd6: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [movieId: int, total_ratings: bigint ... 1 more field]


In [94]:
rdd6.show()

+-------+-------------+------------------+
|movieId|total_ratings|       avg_ratings|
+-------+-------------+------------------+
|    318|          315| 4.450793650793651|
|    527|          213| 4.335680751173709|
|   2959|          215| 4.325581395348837|
|   1221|          127| 4.311023622047244|
|   2571|          269| 4.308550185873606|
|    858|          191| 4.306282722513089|
|    296|          299| 4.285953177257525|
|  48516|          106| 4.283018867924528|
|     50|          202|  4.26980198019802|
|    260|          249| 4.259036144578313|
|   1198|          197| 4.258883248730965|
|   1213|          126|              4.25|
|   4973|          118| 4.241525423728813|
|  58559|          149| 4.238255033557047|
|   1196|          210| 4.233333333333333|
|   1197|          142| 4.232394366197183|
|   1193|          132|4.2272727272727275|
|   1089|          130| 4.226923076923077|
|   1208|          107| 4.219626168224299|
|   2329|          129| 4.217054263565892|
+-------+--

In [98]:
mostPopularMoviesDf.coalesce(1).write.mode("overwrite")
                .option("header", true)
                .csv("hdfs://localhost:9000/sql-popular-movies")

In [99]:
mostPopularMoviesDf.coalesce(1).write.mode("overwrite")
                .option("header", true)
                .json("hdfs://localhost:9000/sql-popular-movies")

In [107]:
val popularMovies = spark.read.format("csv")
                .option("header", true)
                .option("inferSchema", true)
                .load("hdfs://localhost:9000/sql-popular-movies")

popularMovies: org.apache.spark.sql.DataFrame = [{"movieId":318: string, "total_ratings":315: string ... 1 more field]


In [108]:
val popularMovies  = spark.read.format("json")
                .option("header", true)
                .option("inferSchema", true)
                .load("hdfs://localhost:9000/sql-popular-movies")

popularMovies: org.apache.spark.sql.DataFrame = [avg_ratings: double, movieId: bigint ... 1 more field]


In [109]:
 = spark.read.format("csv")
                .option("header", true)
                .option("inferSchema", true)
                .load("hdfs://localhost:9000/sql-popular-movies")

<console>: 31: error: missing argument list for method print in object Predef

In [110]:
spark.sql("""CREATE OR REPLACE TEMP VIEW most_popular_temp_table AS 
                    SELECT movieId, avg(rating) as avg_ratings, 
                                    count(userId) as total_ratings
                        from ratings 
                        WHERE rating > 1.0
                        group by movieId 
                        having avg_ratings >= 3.5 and total_ratings >= 100
                        order by avg_ratings desc
                    """)


res66: org.apache.spark.sql.DataFrame = []


In [111]:
spark.sql("SELECT * FROM most_popular_temp_table").show(4)


+-------+-----------------+-------------+
|movieId|      avg_ratings|total_ratings|
+-------+-----------------+-------------+
|    318|4.450793650793651|          315|
|    527|4.335680751173709|          213|
|   2959|4.325581395348837|          215|
|   1221|4.311023622047244|          127|
+-------+-----------------+-------------+
only showing top 4 rows



In [112]:
val df = spark.table("most_popular_temp_table")
df.printSchema()
df.show()


root
 |-- movieId: integer (nullable = true)
 |-- avg_ratings: double (nullable = true)
 |-- total_ratings: long (nullable = false)

+-------+------------------+-------------+
|movieId|       avg_ratings|total_ratings|
+-------+------------------+-------------+
|    318| 4.450793650793651|          315|
|    527| 4.335680751173709|          213|
|   2959| 4.325581395348837|          215|
|   1221| 4.311023622047244|          127|
|   2571| 4.308550185873606|          269|
|    858| 4.306282722513089|          191|
|    296| 4.285953177257525|          299|
|  48516| 4.283018867924528|          106|
|     50|  4.26980198019802|          202|
|    260| 4.259036144578313|          249|
|   1198| 4.258883248730965|          197|
|   1213|              4.25|          126|
|   4973| 4.241525423728813|          118|
|  58559| 4.238255033557047|          149|
|   1196| 4.233333333333333|          210|
|   1197| 4.232394366197183|          142|
|   1193|4.2272727272727275|          132|
|   108

df: org.apache.spark.sql.DataFrame = [movieId: int, avg_ratings: double ... 1 more field]
